In [4]:
# !git clone https://github.com/dna-witch/KGPL-PyTorch/ -b th-dev
!git clone https://github.com/dna-witch/KGPL-PyTorch/ -b sm-test

Cloning into 'KGPL-PyTorch'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 354 (delta 88), reused 60 (delta 26), pack-reused 169 (from 1)
Receiving objects: 100% (354/354), 15.35 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [5]:
!pip install -r 'KGPL-PyTorch/requirements.txt'

In [6]:
!cd KGPL-PyTorch &&\
python preprocess/preprocess.py -d "music" &&\
python preprocess/make_path_list.py lp_depth=6 dataset=music kg_path=data/music/kg_final.npy rating_path=data/music/ratings_final.npy num_neighbor_samples=32

reading item index to entity id file: data/music/item_index2entity_id.txt ...
reading rating file ...
converting rating file ...
number of users: 1872
number of items: 3846
converting kg file ...
number of entities (containing items): 9366
number of relations: 60
done
adj_entity_path: data/music/adj_entity_6_32.npy
adj_relation_path: data/music/adj_relation_6_32.npy
data_path: data/music/fold1.pkl
dataset: music
kg_path: data/music/kg_final.npy
lp_depth: 6
num_neighbor_samples: 32
pathlist_path: data/music/path_list_6_32.pkl
rating_path: data/music/ratings_final.npy
reachable_items_path: data/music/reachable_items.pkl

[Parallel(n_jobs=32)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Batch computation too fast (0.08682990074157715s.) Setting batch_size=2.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    1.2s
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:    2.4s
[Parallel(n_jobs=32)]: Done  34 tasks      | elapsed:    6.1s
[Pa

In [7]:
import sys
sys.path.append('KGPL-PyTorch/')

In [8]:
from model import *
from utils.data_classes import *
from utils.functions import *
from utils.eval import *

import torch
from tqdm import tqdm

In [9]:
music = KGPLExperiment(
    'music',
    {
      'plabel_lp_depth': 6,
      'plabel_par': 16,
      'plabel_chunk_size': 250,
      'plabel_neg_pn': 0.1,
      'plabel_pl_pn': 1e-3,
      'dropout_rate': 0.5,
      'emb_dim': 64,
      'n_iter': 1,
      'plabel': {},
      'optimize': {'iter_per_epoch': 100, 'lr': 3e-3, 'batch_size': 3333},
      'log': {'show_loss': True},
      'evaluate': {'user_num_topk': 1000},
      'model': {
          'n_iter': 1,
          'neighbor_sample_size': 32,
          'num_neighbor_samples': 32,
          'dropout_rate': 0.5}
    })

Loading Entity Adjacencies...
Loading Relation Adjacencies...
Loading Ratings...
Loading Path List...
Loading Distances...
Setting up dst dict...


100%|██████████| 3846/3846 [00:00<00:00, 7814.27it/s]

Start updating path info...
Path info updated.


In [10]:
music.train_val_test_split()

/content/KGPL-PyTorch/utils/data_classes.py:197: RuntimeWarning: invalid value encountered in divide
  F = (F / F.sum()).cumsum()


Neg C Dict User: 1870
Neg C Dict Item: 3827


100%|██████████| 27097/27097 [00:00<00:00, 115766.13it/s]


Populating item dist dict...


8it [00:06,  1.24it/s]


In [11]:
music.create_dataloaders()

In [12]:
model = KGPLCOT(
        music.cfg,
        int(music.ratings[:, 0].max().item()) + 1,
        int(music.ratings[:, 1].max().item()) + 1,
        music.n_entity,
        music.n_relation,
        music.adj_entity,
        music.adj_relation,
        music.path_list_dict,
        music.train_loader,
        music.val_loader,
        'cpu'
    )

In [13]:
def run_cotrain(model, train_loader, val_loader, test_loader, cfg, device, num_epochs=40, k=10):
    optimizer_f = torch.optim.Adam(model.model_f.parameters(), lr=cfg['optimize']['lr'])
    optimizer_g = torch.optim.Adam(model.model_g.parameters(), lr=cfg['optimize']['lr'])
    iter_per_epoch = cfg['optimize']['iter_per_epoch']

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}")
        model.train()
        total_loss_f, total_loss_g = 0.0, 0.0

        train_iter = iter(train_loader)  # re-initialize iterator every epoch

        for i in tqdm(range(iter_per_epoch)):
            try:
                batch = next(train_iter)
            except StopIteration:
                train_iter = iter(train_loader)
                batch = next(train_iter)

            users, pos, neg, pseudo = [b.to(device) for b in batch]
            batch_data = torch.stack([users, pos, neg, pseudo], dim=1)

            losses = model.train_step(batch_data, optimizer_f, optimizer_g)
            total_loss_f += losses['loss_f']
            total_loss_g += losses['loss_g']

        avg_loss_f = total_loss_f / iter_per_epoch
        avg_loss_g = total_loss_g / iter_per_epoch
        print(f"Train Loss - f: {avg_loss_f:.4f}, g: {avg_loss_g:.4f}")

        print("Evaluating model_f:")

        run_topk_eval(
          model=model,
          cfg = cfg,
          train_data=music.train_dataset.ratings.numpy(),
          eval_data=music.val_dataset.ratings.numpy(),
          test_data=music.test_dataset.ratings.numpy(),
          n_items=music.n_item,
          device=device,
          test_mode=False  # or True for test set
        )

In [14]:
run_cotrain(
    model,
    music.train_loader,
    music.val_loader,
    music.test_loader,
    music.cfg,
    'cpu'
)


Epoch 1


100%|██████████| 100/100 [02:33<00:00,  1.54s/it]


Train Loss - f: 5.0537, g: 5.0519
Evaluating model_f:


Evaluating: 100%|██████████| 1000/1000 [01:20<00:00, 12.35it/s]



Top-K Evaluation Results
K=  1 | Precision: 0.0000 | Recall: 0.0000
K=  2 | Precision: 0.0010 | Recall: 0.0008
K=  5 | Precision: 0.0008 | Recall: 0.0021
K= 10 | Precision: 0.0006 | Recall: 0.0029
K= 20 | Precision: 0.0008 | Recall: 0.0066

Epoch 2


...


Top-K Evaluation Results
K=  1 | Precision: 0.0350 | Recall: 0.0185
K=  2 | Precision: 0.0305 | Recall: 0.0302
K=  5 | Precision: 0.0258 | Recall: 0.0697
K= 10 | Precision: 0.0220 | Recall: 0.1089
K= 20 | Precision: 0.0181 | Recall: 0.1820

Epoch 40


100%|██████████| 100/100 [02:31<00:00,  1.51s/it]


Train Loss - f: 1.8002, g: 1.8079
Evaluating model_f:


Evaluating: 100%|██████████| 1000/1000 [01:13<00:00, 13.62it/s]


Top-K Evaluation Results
K=  1 | Precision: 0.0280 | Recall: 0.0163
K=  2 | Precision: 0.0225 | Recall: 0.0226
K=  5 | Precision: 0.0258 | Recall: 0.0645
K= 10 | Precision: 0.0218 | Recall: 0.1105
K= 20 | Precision: 0.0181 | Recall: 0.1812
